In [0]:
pip install azure-storage-blob

In [0]:
pip install nltk

In [0]:
pip install wordcloud

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords

In [0]:
import requests
import pandas as pd
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.pipeline.transport import HttpResponse
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
import gzip
import plotly
import plotly.graph_objects as go
from pyspark.sql.functions import dayofmonth, countDistinct, to_date, hour
from pyspark.sql.functions import explode
from pyspark.sql.functions import lower
from pyspark.sql.functions import regexp_replace, lower
from pyspark.sql.functions import length
from pyspark.sql.functions import trim


In [0]:
spark = SparkSession.builder.getOrCreate()

storage_account_name = "pod4projectstorage"
storage_account_access_key = "2hClDrVPLGX4QBDBk8OylAkHqczIQfDja66Yl488rmj/0+vb+CAzOxL5qMe5XyM9ZupgwveVRm3N+AStriO5vg=="

container_name = "pod4-project-blob-container"
directory_path = ""

spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)

df = spark.read.parquet("wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/" + directory_path)


In [0]:
df.show()

In [0]:
issues_df = df.filter(df.type=='PushEvent')

In [0]:
for col_name in issues_df.columns:
    new_col_name = col_name.replace("payload.", "")
    issues_df = issues_df.withColumnRenamed(col_name, new_col_name)


In [0]:
issues_df.show()

In [0]:
descrip = issues_df.select(explode('issue.labels').alias("Label"))

In [0]:
descrip = issues_df.select(explode('commits.message').alias("message"))

In [0]:
grouped_df = df.groupBy("type").count().orderBy("count", ascending=False)

In [0]:
pd_data = grouped_df.toPandas()
pd_data

In [0]:
fig = go.Figure([go.Bar(x=pd_data['type'], y=pd_data['count'],marker=dict(color='blue'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Event Counts',xaxis=dict(tickangle=90))
fig.show()

In [0]:
push_df = issues_df

In [0]:
for col_name in df.columns:
    new_col_name = col_name.replace("payload.", "")
    push_df = push_df.withColumnRenamed(col_name, new_col_name)
push_df.show()

In [0]:
hour_df = push_df.withColumn('hour', hour(push_df['created_at']))

In [0]:
hour_df.show()

In [0]:
grouped_hour = hour_df.groupBy("hour").count().orderBy("count", ascending=False)

In [0]:
pd_hour = grouped_hour.toPandas()

In [0]:
fig = go.Figure([go.Bar(x=pd_hour['hour'], y=pd_hour['count'],marker=dict(color='orangered'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig.update_layout(title_text='Commits by Hour',width = 1000, height = 550)
fig.show()

In [0]:
result = push_df.groupBy(dayofmonth("created_at").alias("Day")).agg(countDistinct("actor.id").alias("Users")).orderBy("Users", ascending=False)

In [0]:
result.show()

In [0]:
pd_date = result.toPandas()

In [0]:
fig = go.Figure([go.Bar(x=pd_date['Day'], y=pd_date['Users'], marker=dict(
        color=['yellowgreen' if val == pd_date['Users'].max() else 'royalblue' for val in pd_date['Users']]))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig.update_layout(title_text='Commits by Day of the Month',width = 1000, height = 550)
fig.show()

In [0]:
grouped_pushdf = push_df.select('org.login').dropna()

In [0]:
grouped_pushdf = grouped_pushdf.groupBy("login").count().orderBy("count", ascending=False)

In [0]:
grouped_pushdf.show()

In [0]:
pd_pushdf = grouped_pushdf.toPandas()
pd_pushdf = pd_pushdf.head()

In [0]:
pd_pushdf

In [0]:
fig = go.Figure([go.Bar(x=pd_pushdf['login'], y=pd_pushdf['count'],marker=dict(color='yellowgreen'))])
fig.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.5)
fig.update_layout(title_text='Organizations with highest number of commits',width = 1000, height = 550)
fig.show()

In [0]:
exploded_msg = push_df.select(explode('commits.message').alias("message"))

In [0]:
exploded_msg.dropna()

In [0]:
exploded_msg.show()

In [0]:
exploded_msg = exploded_msg.withColumn("message", trim(lower(regexp_replace(exploded_msg.message, "[^a-zA-Z\\s]", ""))))

In [0]:
exploded_msg = exploded_msg.filter(length(exploded_msg.message) > 0)


In [0]:
exploded_msg.show()

In [0]:
tokenizer = Tokenizer(inputCol="message", outputCol="tokens")
tz_msg = tokenizer.transform(exploded_msg)

In [0]:
tz_msg.show()

In [0]:
remover = StopWordsRemover(inputCol='tokens', outputCol='fil_msg')
tz_msg = remover.transform(tz_msg).drop('tokens')

In [0]:
tz_msg.show()

In [0]:
issue_df = df.filter(df.type == "IssuesEvents")

In [0]:
issue_df.show()